# Combined datasets

<table style="border: 0; border-collapse: collapse; border-spacing: 0;">
  <tr>
    <td style="width:50%; border: none;">
    Problem: Data analysis frequently necessitates the manipulation of datasets scattered across various tables, such as those found in pandas DataFrames or SQL databases. The integration and examination of this distributed data mandate the execution of table joins via shared fields. Although this operation is not inherently difficult, it can become cumbersome when the datasets are extensive or when numerous tables are involved.
    <p><p>
    See more in discussion: <a href="https://github.com/i2mint/tabled/discussions/3">https://github.com/i2mint/tabled/discussions/3</a>
    </td>
    <td style="border: none;">
      <img src="https://github.com/i2mint/tabled/assets/1906276/37dd2e30-3792-4d0b-94a5-c48e83986061" width=320 alt="Venn diagram of fields">
    </td>
  </tr>
</table>

## Making get_table_join

We want to make this:

```python
def get_table_join(tables, fields):
    """
    Get table with requested `fields`, computed by joining relevant tables 
    of `tables`.
    """
    resolution_sequence = join_resolution(tables, fields)
    return compute_join_resolution(resolution_sequence, tables)

# where...

def join_resolution(field_sets: dict, fields_to_cover: Iterable) -> list:
    """
    Returns the list of join operations that, when carried out, 
    cover the given fields.
    
    :param field_sets: A mapping of table names to sets of their fields.
    :param fields: The fields to cover.
    """

def compute_join_resolution(
        resolution_sequence: Iterable, tables: Mapping[str, pd.DataFrame]
    ) -> pd.DataFrame:
    """
    Carries `resolution_sequence` join operations out with tables taken 
    from `tables`.
    
    :param resolution_sequence: An iterable of join operations to carry out. 
        Each join operation is either a table name (str) or a JoinWith object.
        If it's a JoinWith object, it's assumed that the table has already 
        been joined and the fields to remove are in the `remove` 
        attribute of the object.
    :param tables: A mapping of table names to tables (pd.DataFrame)
    """
```

In [91]:
from typing import Callable, Iterable, Mapping
from dataclasses import dataclass
import pandas as pd

import pandas as pd

tables = {
    "A": pd.DataFrame({'b': [1, 2, 3, 33], 'c': [4, 5, 6, 66]}),
    "B": pd.DataFrame(
        {'b': [1, 2, 3], 'a': [4, 5, 6], 'd': [7, 8, 9], 
         'e': [10, 11, 12], 'f': [13, 14, 15]}
    ),
    "C": pd.DataFrame({'f': [13, 14, 15], 'g': [4, 5, 6]}),
    "D": pd.DataFrame(
        {'d': [7, 8, 77], 'e': [10, 11, 77], 'h': [7, 8, 9], 'i': [1, 2, 3]}
    ),
    "E": pd.DataFrame({'i': [1, 2, 3], 'j': [4, 5, 6]})
}

field_sets = {table_id: set(df.columns) for table_id, df in tables.items()}
assert field_sets == {
    "A": {'b', 'c'},
    "B": {'b', 'a', 'd', 'e', 'f'},
    "C": {'f', 'g'},
    "D": {'d', 'e', 'h', 'i'},
    "E": {'i', 'j'}
}

@dataclass
class JoinWith:
    table_key: str
    remove: list = None

fields_to_cover = ['b', 'g', 'j']
expected_join_resolution = [
    'B',
    JoinWith('C', remove=['a', 'f']),
    JoinWith('D', remove=['d', 'e', 'h']),
    JoinWith('E', remove=['i'])
]
expected_result = pd.DataFrame({
    'b': [1, 2],
    'g': [4, 5],
    'j': [4, 5]
})


The tests would be

In [92]:
def test_join_resolution(
        join_resolution: Callable,
        *,
        field_sets: dict = field_sets, 
        fields_to_cover: Iterable = fields_to_cover,
        expected_join_resolution: list = expected_join_resolution,
    ):
    assert join_resolution(field_sets, fields_to_cover) == expected_join_resolution

def test_compute_join_resolution(
    compute_join_resolution: Callable, 
    *,
    resolution_sequence: Iterable = expected_join_resolution,
    tables: Mapping[str, pd.DataFrame] = tables,
    expected_result: pd.DataFrame = expected_result,   
):
    result = compute_join_resolution(resolution_sequence, tables)
    assert tables_are_equal(result, expected_result)


def tables_are_equal(
        t1, t2, *, ignore_index=True, sort_index=True, sort_columns=True
    ):
    """
    Compare two tables for equality, ignoring (by default) index and column order.
    """
    if ignore_index:
        t1 = t1.reset_index(drop=True)
        t2 = t2.reset_index(drop=True)
    elif sort_index:
        t1 = t1.sort_index(axis=1)
        t2 = t2.sort_index(axis=1)
    if sort_columns:
        t1 = t1.sort_index(axis=0)
        t2 = t2.sort_index(axis=0)
    return t1.equals(t2)


In [93]:
def ensure_join_op(obj):
    if not isinstance(obj, JoinWith):
        return JoinWith(obj)
    return obj
        

def compute_join_resolution(
        resolution_sequence: Iterable, tables: Mapping[str, pd.DataFrame]
    ) -> pd.DataFrame:
    """
    Carries `resolution_sequence` join operations out with tables taken from `tables`.
    
    :param resolution_sequence: An iterable of join operations to carry out. 
        Each join operation is either a table name (str) or a JoinWith object.
        If it's a JoinWith object, it's assumed that the table has already been joined
        and the fields to remove are in the `remove` attribute of the object.
    :param tables: A mapping of table names to tables (pd.DataFrame)
    """
    join_ops = map(ensure_join_op, resolution_sequence)
    table_key = next(join_ops).table_key
    joined = tables[table_key]
    for join_op in join_ops:
        table = tables[join_op.table_key]
        joined = joined.merge(table, how='inner')
        if join_op.remove:
            remove_cols = set(join_op.remove) & set(joined.columns)
            joined = joined.drop(columns=remove_cols)
    return joined

test_compute_join_resolution(compute_join_resolution)

**TODO: Implement the join_resolution function -- possibly using the [intersection_graph](https://github.com/i2mint/tabled/blob/282784fc4113b47371ce59a6e14ad2566d7f2abb/tabled/util.py#L25) function on field_sets to help**

## Wip Scrap

In [48]:
def test_join_resolution(
        join_resolution: Callable,
        *,
        field_sets: dict = field_sets, 
        fields_to_cover: Iterable = fields_to_cover,
        expected_join_resolution: list = expected_join_resolution,
    ):
    assert join_resolution(field_sets, fields_to_cover) == expected_join_resolution


# Given test data and expected results
field_sets = {
    "A": {'b', 'c'},
    "B": {'b', 'a', 'd', 'e', 'f'},
    "C": {'f', 'g'},
    "D": {'d', 'e', 'h', 'i'},
    "E": {'i', 'j'}
}
fields_to_cover = ['b', 'g', 'j']


def join_resolution(field_sets: dict, fields_to_cover: Iterable[str]) -> list:
    fields_to_cover = set(fields_to_cover)
    field_sets = field_sets.copy()
    resolution_sequence = []
    # Initialize covered fields with an empty set
    covered_fields = set()

    # Loop until we've covered all fields
    while not fields_to_cover.issubset(covered_fields):
        # Find the table that has the most uncovered fields to cover
        next_table = max(
            field_sets.keys(),
            key=lambda t: len(fields_to_cover.intersection(field_sets[t])) - len(covered_fields.intersection(field_sets[t])),
            default=None
        )

        if next_table is None:
            # If we can't find a next table, it means we can't cover the fields with the given field sets
            raise ValueError("Cannot cover all fields with the given field sets.")

        # Determine which fields from the new table we will need to remove after the join
        fields_to_remove = (covered_fields & field_sets[next_table]) - fields_to_cover

        # If this is not the first table, we create a JoinWith instance, otherwise, just add the table name
        if resolution_sequence:
            resolution_sequence.append(JoinWith(table_key=next_table, remove=list(fields_to_remove)))
        else:
            resolution_sequence.append(next_table)

        # Update covered fields
        covered_fields.update(field_sets[next_table])

        # Remove the used table from field_sets to prevent reusing it
        del field_sets[next_table]

    return resolution_sequence

# test_join_resolution(join_resolution)
join_resolution(field_sets, fields_to_cover)


['A', JoinWith(table_key='C', remove=[]), JoinWith(table_key='E', remove=[])]

In [53]:
# Given test data and expected results
field_sets = {
    "A": {'b', 'c'},
    "B": {'b', 'a', 'd', 'e', 'f'},
    "C": {'f', 'g'},
    "D": {'d', 'e', 'h', 'i'},
    "E": {'i', 'j'}
}
fields_to_cover = ['b', 'g', 'j']

def test_minimum_covering_intersections(
        minimum_covering_intersections: Callable,
        *,
        field_sets: dict = field_sets, 
        fields_to_cover: Iterable = fields_to_cover,
        expected_result: set = {('B', 'C'), ('B', 'D'), ('B', 'E')}
    ):
    assert minimum_covering_intersections(field_sets, fields_to_cover) == expected_result


def minimum_covering_intersections(field_sets, fields_to_cover):
    """Returns a set of set pairs that connect the elements of `fields_to_cover`.
    
    :param field_sets: A mapping of sets of field names
    :param fields_to_cover: A set of field names to cover
    :return: A set of set pairs that connect the elements of `fields_to_cover`
    
    The returned pairs are `(key_1, key_2)` pairs of keys of `field_sets`
    such that field_sets[key_1] & field_sets[key_2] is not empty.
    The union of all the sets in the pairs should cover `fields_to_cover`.
    """
    

from itertools import combinations

def minimum_covering_intersections(field_sets, fields_to_cover):
    fields_to_cover = set(fields_to_cover)  # Ensure we have a set to work with
    covering_pairs = set()
    covered_fields = set()

    # Go through all unique pairs of keys from field_sets
    for key1, key2 in combinations(field_sets.keys(), 2):
        intersection = field_sets[key1] & field_sets[key2]
        
        # Check if this pair covers any new fields that we need to cover
        newly_covered = intersection & fields_to_cover - covered_fields
        if newly_covered:
            covering_pairs.add((key1, key2))
            covered_fields.update(newly_covered)
            
        # If we've covered all fields, we can stop early
        if covered_fields == fields_to_cover:
            break

    return covering_pairs

minimum_covering_intersections(field_sets, fields_to_cover)


{('A', 'B')}

In [76]:
field_sets

{'A': {'b', 'c'},
 'B': {'a', 'b', 'd', 'e', 'f'},
 'C': {'f', 'g'},
 'D': {'d', 'e', 'h', 'i'},
 'E': {'i', 'j'}}

In [80]:
from tabled.util import intersection_graph

assert intersection_graph(field_sets) == {
    'A': {'B'}, 'B': {'A', 'C', 'D'}, 'C': {'B'}, 'D': {'B', 'E'}, 'E': {'D'}
}





In [88]:
from typing import Dict, Tuple, Set

def minimum_covering_intersections(field_sets: Dict[str, Set[str]], fields_to_cover: Set[str]) -> Set[Tuple[str, str]]:
    # Construct the intersection graph
    graph = intersection_graph(field_sets, edge_labels='elements')
    
    # Initialize variables to track covered fields and the pairs to return
    covered_fields = set()
    covering_pairs = set()

    # Function to perform Depth-First Search (DFS) on the graph
    def dfs(node, visited, component):
        visited.add(node)
        component.add(node)
        for neighbour in graph[node]:
            if neighbour not in visited:
                dfs(neighbour, visited, component)
                
    # Find all connected components in the graph
    visited = set()
    components = []
    for node in graph:
        if node not in visited:
            component = set()
            dfs(node, visited, component)
            components.append(component)
    
    # Find the minimum set of set pairs for each component
    for component in components:
        # Check if this component contains any of the fields we need to cover
        if any(field in covered_fields for node in component for field in field_sets[node]):
            # Perform a modified Prim's algorithm to find the minimum spanning tree that covers fields_to_cover
            mst_edges = set()  # This will store the edges of the minimum spanning tree
            component_fields_to_cover = {field for node in component for field in field_sets[node] if field in fields_to_cover}
            covered_in_component = set()
            available_edges = [(len(graph[node][neighbour]), node, neighbour) for node in component for neighbour in graph[node] if neighbour in component]
            sorted_edges = sorted(available_edges)
            
            while component_fields_to_cover - covered_in_component:
                # Select the edge with minimum weight (smallest intersection)
                _, node1, node2 = sorted_edges.pop(0)
                # Add this edge if it helps to cover more fields
                if field_sets[node1] & field_sets[node2] & fields_to_cover:
                    mst_edges.add((node1, node2))
                    covered_in_component.update(field_sets[node1] & field_sets[node2])
            
            # Add the edges of this minimum spanning tree to the covering pairs
            covering_pairs.update(mst_edges)
    
    # Make sure the pairs are in the correct order
    ordered_covering_pairs = {(min(pair), max(pair)) for pair in covering_pairs}
    
    return ordered_covering_pairs


field_sets = {'A': {'b', 'c'},
 'B': {'a', 'b', 'd', 'e', 'f'},
 'C': {'f', 'g'},
 'D': {'d', 'e', 'h', 'i'},
 'E': {'i', 'j'}}

fields_to_cover = ['b', 'g', 'j']

edges = minimum_covering_intersections(field_sets, fields_to_cover)
edges

set()

In [87]:
edges = minimum_covering_intersections(field_sets, fields_to_cover)
assert sorted(edges) == [('B', 'C'), ('B', 'D'), ('D', 'E')]

AssertionError: 

In [28]:
from typing import Iterable, Mapping
from dataclasses import dataclass
import pandas as pd

# Define the JoinWith dataclass
@dataclass
class JoinWith:
    table_key: str
    remove: list = None

# Define the function to find the join resolution
def join_resolution(field_sets: dict, fields_to_cover: Iterable[str]) -> list:
    fields_to_cover = set(fields_to_cover)  # Set of fields we need to cover
    join_sequence = []  # The output sequence of joins
    covered_fields = set()  # Fields covered by the join sequence so far

    # While there are fields left to cover, iterate
    while fields_to_cover:
        # Find the table that covers the most fields which are not yet covered
        next_table, next_fields = max(
            field_sets.items(),
            key=lambda item: len(fields_to_cover.intersection(item[1])) - len(covered_fields.intersection(item[1])),
            default=(None, None)
        )

        # If no table can cover any more fields, break the loop
        if not next_table:
            break

        # Determine which fields we will need to remove after joining this table
        fields_to_remove = (covered_fields | next_fields) - fields_to_cover

        # Add the next table to the join sequence
        if join_sequence:
            join_sequence.append(JoinWith(next_table, remove=list(fields_to_remove)))
        else:  # If this is the first table, we don't need to remove any fields
            join_sequence.append(next_table)

        # Update the sets of covered fields and the remaining fields to cover
        covered_fields.update(next_fields)
        fields_to_cover -= next_fields

        # Remove the chosen table from field_sets to avoid re-selecting it
        del field_sets[next_table]

    # Ensure that the last operation does not include a removal of fields from the last table joined
    if join_sequence and isinstance(join_sequence[-1], JoinWith):
        join_sequence[-1].remove = [f for f in join_sequence[-1].remove if f in covered_fields - fields_to_cover]

    return join_sequence

# Given test data and expected results
field_sets = {
    "A": {'b', 'c'},
    "B": {'b', 'a', 'd', 'e', 'f'},
    "C": {'f', 'g'},
    "D": {'d', 'e', 'h', 'i'},
    "E": {'i', 'j'}
}
fields_to_cover = ['b', 'g', 'j']

# Expected output
expected_join_resolution = [
    'B',
    JoinWith('C', remove=['a', 'f']),
    JoinWith('D', remove=['d', 'e', 'h']),
    JoinWith('E', remove=['i'])
]

# Run the function with the test data
join_resolution_output = join_resolution(field_sets.copy(), fields_to_cover)

# Display the output
join_resolution_output


['A',
 JoinWith(table_key='C', remove=['f', 'c', 'b']),
 JoinWith(table_key='E', remove=['g', 'i', 'f', 'b', 'c'])]

# Proposal: tiny light framework for routing with mappings

In [10]:
from i2.routing_forest import KeyFuncMapping
from tabled import dflt_ext_mapping, get_ext

from dol import Files, FilesOfZip, Pipe, wrap_kvs
import pandas as pd
from functools import partial
import io

ext = KeyFuncMapping(
    dict(
        dflt_ext_mapping, 
        json=Pipe(io.BytesIO, partial(pd.read_json, orient='index'))
    ),
    get_ext,
)

table_trans = wrap_kvs(postget=lambda k, v: ext[k](v))

TableFiles = table_trans(Files)
TableZipFiles = table_trans(FilesOfZip)

In [11]:
s = TableZipFiles('/Users/thorwhalen/Dropbox/_odata/sound/induction_motor_data.zip')

In [12]:
len(s)

3909

In [67]:
k, v = s.head()
print(f"{k=}")
v

k='induction_motor_data/Bearing/Bearing_1250rpm/2021_01_27_15_03_06.json'


,0
dataType,completeSample
deviceId,00000781O
flux,"[31.3639, 23.8975, 35.5119, 27.0085, 16.6385, ..."
motorId,adi#0c7dbd85-a5e7-4e6a-b37c-a49e1adaca45
tempe,"[25.875, 25.875]"
tempm,"[25.9375, 25.9375]"
tenantId,adi
timestamp,1611756186321
ts,60814
tsr,False


In [20]:
from hubcap import GitHubReader, Github


t = GitHubReader('i2mint')
tt = list(t);

In [26]:
w = filter(lambda x: x.endswith('dol'), tt)
w = map(lambda x: f"[{x}](https://www.github.com/i2mint/{x})", w)
print(*w, sep='\n    * ')

[aiofiledol](https://www.github.com/i2mint/aiofiledol)
    * [arangodol](https://www.github.com/i2mint/arangodol)
    * [couchdol](https://www.github.com/i2mint/couchdol)
    * [dol](https://www.github.com/i2mint/dol)
    * [dropboxdol](https://www.github.com/i2mint/dropboxdol)
    * [dynamodol](https://www.github.com/i2mint/dynamodol)
    * [ftpdol](https://www.github.com/i2mint/ftpdol)
    * [mongodol](https://www.github.com/i2mint/mongodol)
    * [odbcdol](https://www.github.com/i2mint/odbcdol)
    * [pydrivedol](https://www.github.com/i2mint/pydrivedol)
    * [redisdol](https://www.github.com/i2mint/redisdol)
    * [s3dol](https://www.github.com/i2mint/s3dol)
    * [sqldol](https://www.github.com/i2mint/sqldol)
    * [sshdol](https://www.github.com/i2mint/sshdol)


In [27]:
from redisdol import RedisBytesPersister

In [30]:
s = RedisBytesPersister()

In [31]:
list(s)

[]

In [1]:
t = [1, 2, 3, 4]

In [2]:
t += [5, 6]
# t.extend([5, 6])
t


[1, 2, 3, 4, 5, 6]

In [3]:
t.append(8)
t

[1, 2, 3, 4, 5, 6, 8]

In [ ]:
t.append

In [ ]:
t[k].append

NameError: name 'pwd' is not defined

In [ ]:
pwd
